In [2]:
import requests
import matplotlib.pyplot as plt
import numpy as np
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from geopy import distance

In [3]:

def get_street_points(street_name,bbox):
    overpass = Overpass()
    query = overpassQueryBuilder(bbox=bbox,
    elementType='way',selector=f'"name"="{street_name}"')

    road_points = []
    for e in overpass.query(query).elements():
        for node in e.nodes():
            road_points.append((node.lat(),node.lon()))

    return road_points

            

In [4]:
SHEFFIELD_BBOX = [53.356987, -1.510101, 53.402656, -1.433196]
road_points = get_street_points("Abney Street", SHEFFIELD_BBOX)


In [5]:
def middle(points):
    return (max(points) + min(points)) / 2


In [15]:
SIM_SIZE = 200
lats = [x for x,_ in road_points] #geo x's
lons = [y for _,y in road_points] #geo y's
geo_center = np.array((middle(lats),middle(lons)))

sim_points = []

for geo_point in road_points:
    geo_point = np.array(geo_point)

    relative_to_geo_center = geo_point-geo_center
    dist = distance.distance(geo_center, geo_point).m
    theta = distance.atan2(relative_to_geo_center[1], relative_to_geo_center[0])

    sim_x = dist * np.cos(theta)
    sim_y = dist * np.sin(theta)
    
    sim_x += SIM_SIZE/2
    sim_y += SIM_SIZE/2

    sim_points.append((sim_x, sim_y))

In [14]:
print(sim_points)

[(44.08150011595894, 97.9919846356238), (83.86844279839524, 85.73576131681948), (126.7468314205182, 65.82948775929052), (133.8058019734039, 65.47346982364496), (149.646918113508, 118.17159582290294), (148.57761607896407, 138.25614110177915)]
